In [1]:
import numpy as np
import cv2
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

def grid(x_range, y_range, delta):
    nodes = []
    square_ranges = []

    # Calculate the step size for each dimension
    x_step = x_range / delta
    y_step = y_range / delta

    for row in range(delta):
        for column in range(delta):
            # Calculate the coordinates of the center of the grid square
            x_coord = (row + 0.5) * x_step
            y_coord = (column + 0.5) * y_step
            nodes.append([x_coord, y_coord])

            # Define the ranges for each square
            x_min = row * x_step
            x_max = (row + 1) * x_step
            y_min = column * y_step
            y_max = (column + 1) * y_step

            square_ranges.append({
                'x_min': x_min,
                'x_max': x_max,
                'y_min': y_min,
                'y_max': y_max
            })

    return nodes, square_ranges

def find_focal_cells(nodes, particles_matrix):
    focal_cells = []

    for node in nodes:
        node_x, node_y = node
        distances = np.sqrt((particles_matrix['x'] - node_x) ** 2 + (particles_matrix['y'] - node_y) ** 2)
        closest_particle_index = np.argmin(distances)
        focal_cells.append(particles_matrix.iloc[closest_particle_index])

    focal_cells_df = pd.DataFrame(focal_cells)
    return focal_cells_df, particles_matrix

def assign_particles_to_grid(frame_data, nodes, square_ranges):
    particle_assignments = pd.DataFrame(index=frame_data.index)

    for i, node in enumerate(nodes):
        node_x, node_y = node
        current_range = square_ranges[i]

        within_boundaries = (
            (frame_data['x'] >= current_range['x_min']) & (frame_data['x'] < current_range['x_max']) &
            (frame_data['y'] >= current_range['y_min']) & (frame_data['y'] < current_range['y_max'])
        )

        particle_assignments.loc[within_boundaries, 'grid_square'] = i

    # Merge the particle_assignments DataFrame back to frame_data
    frame_data = pd.merge(frame_data, particle_assignments, left_index=True, right_index=True, how='left')

    return frame_data

def calculate_nematic_order(focal_cells_df, particles_matrix, grid_indices):
    nematic_orders_array_1 = []
    nematic_orders_array_2 = []

    # Loop over each frame
    for frame_num, frame_data in particles_matrix.groupby('frame'):
        focal_vectors = np.zeros((len(focal_cells_df), 2))
        focal_vectors[:, 0] = np.cos(focal_cells_df['body_angle'].values)
        focal_vectors[:, 1] = np.sin(focal_cells_df['body_angle'].values)
        
        # Loop through the selected grid squares
        for i, square_index in enumerate(grid_indices):
            square_data = frame_data[frame_data['grid_square'] == square_index]
            body_angle_vector = np.array([
                np.cos(square_data['body_angle'].values),
                np.sin(square_data['body_angle'].values)
            ]).T
            current_focal = np.zeros((len(body_angle_vector), 2))
            current_focal[:, :] = focal_vectors[square_index, :]
            

            # Calculate dot products for each focal cell with body angle vectors in the selected grid square
            dot_products = np.dot(current_focal, body_angle_vector.T)

            # Calculate nematic orders
            print(dot_products)
            nematic_order = (3/2 * dot_products**2 - 0.5).mean()

            # Append nematic orders to the appropriate array
            if i == 0:
                nematic_orders_array_1.append(nematic_order)
            elif i == 1:
                nematic_orders_array_2.append(nematic_order)

    return nematic_orders_array_1, nematic_orders_array_2

# Import data from your directory
df = pd.read_csv(r"/Users/Cameron/Documents/Fall2023/Mechanobiology/Project1/trajectory_WT_dilute.csv")

# Defining range of the image grid formation
max_x = df['x'].max()
max_y = df['y'].max()

# Calculate nematic orders for two specific grid squares
nodes, square_ranges = grid(max_x, max_y, 5)
focal_cells_df, frame_data = find_focal_cells(nodes, df)

# Specify the indices of the two grid squares you want to analyze
selected_grid_indices = [3, 23]  # Change these indices as needed
frame_data = assign_particles_to_grid(frame_data, nodes, square_ranges)

nematic_orders_array_1, nematic_orders_array_2 = calculate_nematic_order(focal_cells_df, frame_data, selected_grid_indices)

# Plot the results for each selected grid square
frame_count_WT_dense = np.unique(df['frame']).size
time = np.linspace(0, frame_count_WT_dense * 3, frame_count_WT_dense)

# Plot the results for each selected grid square
plt.scatter(time, nematic_orders_array_1, marker='o', label=f'Grid Square {selected_grid_indices[0]}')
plt.scatter(time, nematic_orders_array_2, marker='o', label=f'Grid Square {selected_grid_indices[1]}')

plt.xlabel('Time [s]')
plt.ylabel('Nematic Order')
plt.ylim(0,1)
plt.legend()
plt.title('Nematic Order over Time for Selected Grid Squares in WT (Dilute)')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'example_tracking_data.csv'